In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from implementations import *
from GAN_function import *

In [ ]:

def GetEvent_Model(P:Particle, table_of_p):
    # Compute probability of particles
    table = table_of_p.copy(deep=True)
    KinE = P.ene
    prob_all = table.loc[(table['Energy_min'] <  KinE) & (table['Energy_max'] >= KinE)]
    prob0, prob1 = prob_all[['proba_0', 'proba_1']].values
    s = random.random() # simple event generator for testing
    if s <= prob0:
        name_s = 0
    elif ((s > prob0) & (s < prob0 + prob1)):
        name_s = 1
    else:
        name_s = 2
    
    # Now we have the type of the particle
    # Let's find the model necessary for the particle
    
    model = get_model(KinE=KinE, name_s=name_s)
    event = model.Get_GAN_event
    
    # We store the data generated
    if name_s!=0:
        #{'cos_theta': 0, 'dE': 1, 'cos_phi': 2, 'cos_psi': 3, 'E_s': 4}
        cos_theta=event[0]
        dE=event[1]
        DX_Q
        DY_Q
        DZ_Q
        EQ=event[4]
    
    
    if name_s!=0:
        if (name_s == 1):
            name_s = Type.electron
        else:
            name_s = Type.photon
        Q = Particle(P.pos[0], P.pos[1], P.pos[2], DX_Q, DY_Q, DZ_Q, EQ, name_s)
        Q.Rotate(0.5)
        return distance, dE, cos_theta, Q
    else:
        return distance, dE, cos_theta, None
    



In [ ]:
WX,WY,WZ = 300,200,200 # 300x200x200 mm
NX,NY,NZ = 150,100,100 # for 2x2x2 mm voxels
A = Arena(WX, WY, WZ, NX, NY, NZ)
NMC = 10000 # number of particles we want to simulate
EMAX = 20.0 # maximum energy of particles

ToSimulate = queue.Queue() # the queue of particles
for i in range(NMC) :
    # create NMC particles at x=0, y=WY/2, z=WZ/2, going in the X direction (1,0,0) and with a gamma distributed energy
    s = random.gauss(0.0, 0.1)
    ypos = (1+s)*WY/2
    ydir = 0.2*s
    s = random.gauss(0.0, 0.1)
    zpos = (1+s)*WZ/2
    zdir = 0.2*s
    xdir = math.sqrt(1 - ydir**2 - zdir**2)
    e = EMAX*random.gammavariate(2.0, 0.667)
    P = Particle(0.0, ypos, zpos, xdir, ydir, zdir, e, Type.electron, True)
    ToSimulate.put(P)

DONE = 0 # count number of primary particles
NALL = 0 # count total number of particles
while not ToSimulate.empty() > 0 :
    P = ToSimulate.get()
    if P.is_primary :
        DONE += 1
    NALL += 1
    while P.ene > 0.0 :
        distance, delta_e, cos_theta, generated_particle = GetEvent(P)
        P.Move(distance)
        P.Lose(delta_e, A)
        P.Rotate(cos_theta)
        if generated_particle is not None :
            ToSimulate.put(generated_particle)
    if DONE % 1000 :
        sys.stdout.write(f"Finished {DONE:8} out of {NMC:8} {(100.0*DONE)/NMC:.2f} %\r"); sys.stdout.flush()
sys.stdout.write(f"\nFinished with {NMC:8} primaries and a total of {NALL:8} particles\n"); sys.stdout.flush()

fig,axes = plt.subplots(nrows=1, ncols=3, figsize=(18,6))
CNTR = axes[0].contourf( np.log10(A.M.sum(axis=1)+0.001), cmap="inferno" )
axes[0].axvline(NY/2, ls=":", color="white")
axes[0].axhline(5, ls=":", color="green")
plt.colorbar(CNTR, aspect=60)
axes[0].set_title("log(Dose) in X/Z plane")
axes[1].plot( A.M[:,int(NY/2),int(NZ/2)] )
axes[1].set_title("dose along white line")
axes[2].plot( A.M[5,:,int(NZ/2)] )
axes[2].set_title("dose across green line")
plt.show()



a = 1
if a != None:
    print(a)
        